In [16]:
from mistralai import Mistral
import os
import pandas as pd
import json

api_key = "JLG3v61hGXuzrpfWB7THHFVAQHDEvdcs"

client = Mistral(api_key=api_key)

ultrachat_chunk_train = client.files.upload(file={
    "file_name": "on_video_train.jsonl",
    "content": open("on_video_train.jsonl", "rb"),
})
ultrachat_chunk_eval = client.files.upload(file={
    "file_name": "on_video_eval.jsonl",
    "content": open("on_video_eval.jsonl", "rb"),
})

In [17]:
print(ultrachat_chunk_eval, ultrachat_chunk_train)

id='959134cd-9156-491c-9295-46548d57fd2e' object='file' bytes=8272 created_at=1728791677 filename='on_video_eval.jsonl' sample_type='instruct' source='upload' PURPOSE='fine-tune' num_lines=10 id='8ad65426-1c2a-435a-b3a9-0a71d8d68651' object='file' bytes=75007 created_at=1728791677 filename='on_video_train.jsonl' sample_type='instruct' source='upload' PURPOSE='fine-tune' num_lines=97


In [18]:
# create a fine-tuning job
created_jobs = client.fine_tuning.jobs.create(
    model="mistral-large-latest", 
    training_files=[{"file_id": ultrachat_chunk_train.id, "weight": 1}],
    validation_files=[ultrachat_chunk_eval.id], 
    hyperparameters={
        "training_steps": 5,
        "learning_rate":0.0001
    },
    auto_start=False
)

In [19]:
# start a fine-tuning job
client.fine_tuning.jobs.start(job_id = created_jobs.id)
created_jobs

JobOut(id='3d8a5d94-d73e-4de7-b3ce-a0546f9bba05', auto_start=False, hyperparameters=TrainingParameters(training_steps=5, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, fim_ratio=None), model='mistral-large-latest', status='QUEUED', job_type='FT', created_at=1728791692, modified_at=1728791692, training_files=['8ad65426-1c2a-435a-b3a9-0a71d8d68651'], validation_files=['959134cd-9156-491c-9295-46548d57fd2e'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None))

In [20]:
created_jobs

JobOut(id='3d8a5d94-d73e-4de7-b3ce-a0546f9bba05', auto_start=False, hyperparameters=TrainingParameters(training_steps=5, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, fim_ratio=None), model='mistral-large-latest', status='QUEUED', job_type='FT', created_at=1728791692, modified_at=1728791692, training_files=['8ad65426-1c2a-435a-b3a9-0a71d8d68651'], validation_files=['959134cd-9156-491c-9295-46548d57fd2e'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None))

In [28]:
# Retrieve a jobs
retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
print(retrieved_jobs)

id='3d8a5d94-d73e-4de7-b3ce-a0546f9bba05' auto_start=False hyperparameters=TrainingParameters(training_steps=5, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=69.90133859527492, fim_ratio=None) model='mistral-large-latest' status='SUCCESS' job_type='FT' created_at=1728791692 modified_at=1728792168 training_files=['8ad65426-1c2a-435a-b3a9-0a71d8d68651'] validation_files=['959134cd-9156-491c-9295-46548d57fd2e'] OBJECT='job' fine_tuned_model='ft:mistral-large-latest:64a2499b:20241013:3d8a5d94' suffix=None integrations=[] trained_tokens=1310720 repositories=[] metadata=JobMetadataOut(expected_duration_seconds=365, cost=11.54, cost_currency='EUR', train_tokens_per_step=262144, train_tokens=1310720, data_tokens=18751, estimated_start_time=None) events=[EventOut(name='status-updated', created_at=1728792168, data={'status': 'SUCCESS'}), EventOut(name='status-updated', created_at=1728791703, data={'status': 'RUNNING'}), EventOut(name='status-updated', created_at=1728791699

In [29]:
chat_response = client.chat.complete(
    model=retrieved_jobs.fine_tuned_model,
    messages = [{"role":'user', "content":'What is the best French cheese?'}]
)

In [30]:
retrieved_jobs.fine_tuned_model

'ft:mistral-large-latest:64a2499b:20241013:3d8a5d94'

In [31]:
chat_response

ChatCompletionResponse(id='650ac712ff354a7887978c65e8110c6e', object='chat.completion', model='ft:mistral-large-latest:64a2499b:20241013:3d8a5d94', usage=UsageInfo(prompt_tokens=10, completion_tokens=314, total_tokens=324), created=1728792308, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='Choosing the "best" French cheese is subjective, as it depends on personal taste. France is renowned for its wide variety of cheeses, with over 400 different types. Here are a few highly regarded ones:\n\n1. **Brie de Meaux**: A soft, creamy cheese made from cow\'s milk. It\'s often considered one of the finest cheeses in the world.\n\n2. **Camembert de Normandie**: Another soft, creamy cheese made from cow\'s milk. It has a stronger flavor than Brie.\n\n3. **Roquefort**: A sheep milk cheese from the south of France. It\'s known for its tangy, salty flavor and distinctive veins of blue mold.\n\n4. **Comté**: A firm, nutty cheese made from cow\'s milk in the Jura region. It\'